In [66]:
app_name="05-colaberry-datadoc"
datadoc_name="regression"
core_template_file = 'dash_template-core.ipynb'
datadoc_title='Predicting Sales using Regression'

In [67]:
import json

#input_content_file = 'input-notebook.ipynb'
input_content_file = 'datadoc-regression-input-new.ipynb'

parsed_json = ''
with open(input_content_file) as json_data:
    parsed_json = json.load(json_data)
#print(parsed_json)
template_file = 'dash_template-regression.ipynb'
template_json = ''
with open(template_file) as template_data:
    template_json = json.load(template_data)
    #print(template_json)
    
core_template_json = ''
with open(core_template_file) as core_template_data:
    core_template_json = json.load(core_template_data)
    #print(template_json)

In [68]:
text_contents = []
code_content = ''
plot_content = ''
dependencies = []
cells = parsed_json['cells']
group_content = 'groups = []' + '\n' + 'registry = {}'

for cell in cells:
    print(cell['metadata']['tags'][0] + ':::' + cell['metadata']['tags'][1])
    #print(cell)
    #print(""               "")
    cell_type = cell['cell_type']
    if cell_type == 'markdown':
        text_content = ''
        text_cell = cell['source']
        for text_line in text_cell:
            text_content = text_content + text_line
        text_contents.append(text_content)
        
    if 'metadata' not in cell:
        #raise ValueError("No metadata found")
        print("No metadata found")

#    if cell_type == 'code' and cell['metadata']['tags'][0] == 'plot':
    if cell_type == 'code' and cell['metadata']['tags'][0] == 'code' and cell['metadata']['tags'][1].startswith('common'):
        plot_content = plot_content + '\n'  
        plot_cell = cell['source']
        for plot_line in plot_cell:
            if not (plot_line.startswith('iplot') or plot_line.startswith('from plotly.offline') or plot_line.startswith('init_notebook_mode')):
                plot_content = plot_content + plot_line
                if plot_line.startswith('import'):
                    dependencies.append(plot_line.split()[1])
            if plot_line.startswith('plt.savefig'):
                plot_content = plot_content + "\n" + "figures.append(figname +'.png')"  + "\n"
                
    if cell['metadata']['tags'][1].startswith('group'):

        plot_content = plot_content + '\n'  
        class_sub_string = 'class '+ cell['metadata']['tags'][1] + ':'
        content_sub_string = '###***content '+ cell['metadata']['tags'][1]
        if class_sub_string not in plot_content:
            plot_content = plot_content + class_sub_string + '\n' + content_sub_string
            group_content = group_content  + '\n' + "groups.append('{}')".format(cell['metadata']['tags'][1])
            group_content = group_content  + '\n' + "registry['{}'] = {}".format(cell['metadata']['tags'][1],cell['metadata']['tags'][1])
            
        if cell_type == 'markdown':
            text_content = ''
            text_cell = cell['source']
            for text_line in text_cell:
                text_content = text_content + text_line
            markdown_content = "text_content = '''{}'''".format(text_content)
            plot_content = plot_content.replace('###***content '+ cell['metadata']['tags'][1],"    " +  markdown_content + '\n'  + '###***content '+ cell['metadata']['tags'][1])


        if cell_type == 'code' and cell['metadata']['tags'][0] == 'plot':
        
            plot_cell = cell['source']
            for plot_line in plot_cell:
                if not (plot_line.startswith('iplot') or plot_line.startswith('from plotly.offline') or plot_line.startswith('init_notebook_mode')):
                    #plot_content = plot_content + plot_line
                    if plot_line.startswith('import'):
                        dependencies.append(plot_line.split()[1])
                    if plot_line.startswith('plt.savefig'):
                        temp_plot_content = "\n" + "figures.append(figname +'.png')"  + "\n"
                    plot_content = plot_content.replace('###***content '+ cell['metadata']['tags'][1],"    " +  plot_line  + '###***content '+ cell['metadata']['tags'][1])

plot_content = plot_content + '\n' + group_content
                
dep = []
for dependency in dependencies:
    dep.append(dependency.split('.')[0])
    unique_dep = set(dep)


#markdown_content = "text_content = []"

#for t in text_contents:
#    markdown_content = markdown_content  + '\n' + "text_content.append('''{}''')".format(t)
#print(plot_content)

desc:::group1
desc:::group2
code:::common
desc:::group3
plot:::group3
desc:::group4
code:::common
plot:::group5


In [69]:
core_template_content = ''
gitignore_content = ''
procfile_content = ''
core_template_cells = core_template_json['cells']
for cell in core_template_cells:
    cell_type = cell['cell_type']
    if cell_type == 'code':
        core_template_cell = cell['source']
        for core_template_line in core_template_cell:
            core_template_content = core_template_content + core_template_line
    if cell_type == 'markdown' and cell['metadata']['tags'][0] == 'git':
        git_cell = cell['source']
        for git_line in git_cell:
            gitignore_content = gitignore_content + git_line
    if cell_type == 'markdown' and cell['metadata']['tags'][0] == 'proc':
        proc_cell = cell['source']
        for proc_line in proc_cell:
            procfile_content = procfile_content + proc_line


In [70]:
template_content = ''
template_cells = template_json['cells']
for cell in template_cells:
    cell_type = cell['cell_type']
    if cell_type == 'code':
        template_cell = cell['source']
        for template_line in template_cell:
            template_content = template_content + template_line

#print(template_content)
#print('  ')
#print(gitignore_content)
#print('   ')
#print(procfile_content)


In [71]:
from pathlib import Path
is_new_app = False
my_file = Path(app_name)
if my_file.is_dir():
    %cd {app_name}
    print('Diectory exists')
else:
    print('Diectory does not exists')
    is_new_app = True
    !mkdir {app_name}
    %cd {app_name}


Diectory does not exists
/home/mani/workspace/dashframework/05-colaberry-datadoc


In [72]:
core_content = ''
app_content = ''

if is_new_app:
    core_content = core_template_content
else:
    with open('app.py') as core_file_data:
        app_content = core_file_data.read() 
    core_content = app_content
    
#print(core_content)
core_content = core_content.replace('###***Import***###',"import " + datadoc_name + "\n" + "###***Import***###")
core_content = core_content.replace('###***Menu***###',"dcc.Link('" + datadoc_name.capitalize() + "   ', href='/" + datadoc_name + "', className='tab')," + "\n" + "        ###***Menu***###")
core_content = core_content.replace('###***Path***###',"elif pathname == '/" + datadoc_name + "':" + "\n" + "        return " + datadoc_name + ".layout" + "\n" + "    ###***Path***###")

app_fname = 'app.py'
core_data = core_content

with open(app_fname, 'w') as f:
    f.write('{}'.format(core_data))

with open('.gitignore', 'w') as f:
    f.write('{}'.format(gitignore_content))

with open('Procfile', 'w') as f:
    f.write('{}'.format(procfile_content))



In [73]:
#creating the app.py file, the .gitignore and Procfile for deployment to heroku
        

    
fname = datadoc_name + '.py'
title_content = "datadoc_title = '{}'".format(datadoc_title)
data = code_content + "\n" + markdown_content + "\n" + plot_content  + "\n" + title_content + '\n' + template_content

with open(fname, 'w') as f:
    f.write('{}'.format(data))


%cd ..

/home/mani/workspace/dashframework


In [23]:
#Actual deployment using deploy.sh
#Requirement - need heroku cli installed and configured locally for this step to work.
args = ''
for each_dep in unique_dep:
    args = args + ' ' + each_dep
if is_new_app:
    print("New App - Deploying for the first time")
    !sh deploy.sh {app_name} {args}
else:
    print("Existing App - Re-Deploying")
    !sh redeploy.sh {app_name} {args}


/home/mani/workspace/dashframework
Existing App - Re-Deploying
Invalid requirement: '='
= is not a valid operator. Did you mean == ?


Skipping pkg-resources as it is not installed.
On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   app.py

no changes added to commit (use "git add" and/or "git commit -a")
[master 382168c] Updated App
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 357 bytes | 357.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Compressing source files... done.
remote: Building source:
remote: 
remote: -----> Python app detected
remote:  !     Python has released a security update! Please consider upgrading to python-3.6.7
remote:        Learn More: https://devcenter.heroku.com/articles/python-runtimes
remote: -----> Installing requirements with pip
remote: 
remote: -----> Discovering process types
remote:     